In [ ]:
%config IPCompleter.greedy=True

import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

# Vérification des versions installées

In [ ]:
print(tf.version.VERSION)

In [ ]:
!dpkg -l | grep nvinfer

# Création du dataset d'entrées

On crée les dataset au format suivant :

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/datset_regression_heatmap2.png?raw=true" width=400>

In [ ]:
repertoire_courant = os.getcwd()

In [ ]:
TACHE = "visage"
CATEGORIES = ['nez']
batch_size = 1

dataset_base = tf.keras.preprocessing.image_dataset_from_directory(
    repertoire_courant+"/projet_regression/"+TACHE,
    validation_split=0.0,
    batch_size=batch_size,
    label_mode="categorical")

In [ ]:
repertoires_images = dataset_base.file_paths

In [ ]:
from imgaug.augmentables.heatmaps import HeatmapsOnImage

def CreateHeatmap(x, y, x0, y0, width, height):
    alpha = 0.4
    sigma_x = alpha*width/6.0
    sigma_y = height/6.0
    return np.exp(-((x - x0)**2. / (2. * sigma_x**2.) + (y - y0)**2. / (2. * sigma_y**2.)))

def CreationDatasetRegression(liste_fichiers,noms_classes,width,height):
    heatmap_ = [[] for i in range(len(noms_classes))]
    images_= []

    width = width
    height = height

    # Création du heatmap nul
    heatmap_zero = np.zeros((height,width)).astype(np.float32)
    heatmap_zero = HeatmapsOnImage(heatmap_zero, shape=(height,width), min_value=0.0, max_value=1.0)
    heatmap_zero = heatmap_zero.avg_pool(4)

    # Création de la grille pour le heatmap
    x,y = np.meshgrid(np.linspace(0,width-1,width), np.linspace(0,height-1,height))
    
    # Définition des constantes pour le heatmap
    alpha = 0.4
    sigma_x = alpha*width/6.0
    sigma_y = alpha*height/6.0
    
    # Conversion des classe du heatmap zero en tenseur 1D
    # (56,56,nbr_classes)
    heatmap_zero = tf.convert_to_tensor(heatmap_zero.get_arr())

    for fichier in liste_fichiers:
        # Chargement de l'image
        image = tf.keras.preprocessing.image.load_img(fichier)
        image = tf.keras.preprocessing.image.img_to_array(image)

        # Extraction des coordonnées (x0,Y0)
        # correspondantes aux dimensions de l'image chargée (height,width)
        element = tf.strings.split(fichier,sep="_image_")
        element = tf.strings.split(element[1],sep="_")
        x0 = tf.strings.to_number(element[1],out_type=tf.dtypes.float32)
        y0 = tf.strings.split(element[2],sep=".")
        y0 = tf.strings.to_number(y0[0],out_type=tf.dtypes.float32)
        
        # Extraction de la classe
        classe = tf.strings.split(fichier,sep="/")[-2]
        
        # Extraction du label
        label = tf.cast(tf.strings.regex_full_match(noms_classes, classe),dtype="int32")
        
        # Extraction de la valeur binaire du label
        maxi = tf.math.argmax(label,output_type=tf.dtypes.int32)
        
        # Création du heatmap de l'image
        heatmap = CreateHeatmap(x,y,x0,y0,width,height).astype(np.float32)
        heatmap = HeatmapsOnImage(heatmap, (height,width,1), min_value=0.0, max_value=1.0)
        heatmap = heatmap.max_pool(4)
        
        # Sauvegarde du heatmap et de l'image dans les listes
        for i in range(len(noms_classes)):
            if i == maxi:
                heatmap = tf.convert_to_tensor(heatmap.get_arr())
                heatmap_[i].append(heatmap)
            else:
                heatmap_[i].append(heatmap_zero)
        images_.append(image)
   
    # Création du dataset
    images_ = tf.convert_to_tensor(images_)                                # (nbr_images,H,W,3)
    heatmap_ = tf.convert_to_tensor(heatmap_)                              # (nbr_images,H/4,W/4,1)
    
    heatmap_ = tf.transpose(heatmap_,perm=[1,2,3,0,4])                     # (nbr_image,H/4,W/4)
    heatmap_ = tf.squeeze(heatmap_,-1)                                     # (nbr_image,H/4,W/4)

    datasetHeatmap = tf.data.Dataset.from_tensors(heatmap_)                # (nbr_images,3,H/4,W/4)
    datasetImg = tf.data.Dataset.from_tensors(images_)                     # (nbr_images,H,W,3) 
    dataset = tf.data.Dataset.zip((datasetImg,datasetHeatmap))

    return (dataset)

In [ ]:
dataset_regression = CreationDatasetRegression(repertoires_images,dataset_base.class_names,width=224, height=224)

In [ ]:
for image,heatmap in dataset_regression.take(1):
    print(image.shape)
    print(heatmap.shape)

# Conversion du modèle sauvegardé au format ONNX

Nous aurons besoin par la suite du modèle au format ONNX pour réaliser les optimisations.

In [ ]:
!python3 -m tf2onnx.convert --saved-model "Regression_Resnet18_saved_model" --output "/home/alexandre/Regression_Resnet18_saved_model/Regression_Resnet18_saved_model.onnx"

On modifie maintenant le batch_size :

In [ ]:
import onnx

batch_size = 1
onnx_model = onnx.load("/home/alexandre/Regression_Resnet18_saved_model/Regression_Resnet18_saved_model.onnx")

In [ ]:
inputs = onnx_model.graph.input

In [ ]:
inputs

In [ ]:
inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = batch_size

Puis on sauvegarde les modifications :

In [ ]:
onnx.save_model(onnx_model,"/home/alexandre/Regression_Resnet18_saved_model/Regression_Resnet18_saved_model.onnx")

### Redémarrage du kernel

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

# Analyse des performances du modèle non optimisé

### Chargement du modèle non optimisé

Chargeons maintenant notre modèle Tensorflow d'origine :

In [ ]:
 def ChargementModele(repertoire):
    print("Chargement du modèle %s ... " %repertoire)
    model = tf.saved_model.load(repertoire)
    return model

In [ ]:
saved_model = ChargementModele('Regression_Resnet18_saved_model')

### Chargement des signatures du modèle

On peut afficher la liste des signatures contenues dans le modèle chargé :

In [ ]:
print(list(saved_model.signatures))

On charge ensuite la signature :

In [ ]:
infer = saved_model.signatures['serving_default']

Ceci nous permet par exemple de regarder le format de la sortie :

In [ ]:
infer.structured_outputs

### Analyses des temps de calculs et du débit de traitement des images

Nous allons utiliser notre modèle pour mesurer son temps de calcul et son débit pour traiter les images. Du fait de l'initialisation du GPU, nous allons mesurer ces caractéristiques après avoir utilisé le modèle un petit nombre de fois :

In [ ]:
import time

def MesureDesPredictions(dataset,infer,nbr_run):
    delais = []
    predictions = []
    
    # Récupère le premier batch
    for images,heatmaps in dataset.take(1):
        batch_input = images
        batch_input = tf.expand_dims(batch_input,0)
        print("Format de l'entrée : %s" %(batch_input.shape))
    
    # Initialisation des calculs
    print("Initialisation des calculs...")
    for i in range(5):
        prediction = infer(batch_input)['heatmap'].numpy()
        predictions.append(prediction)
    
    # Lance les inférences
    for i in range(nbr_run):
        time0 = time.time()
        prediction = infer(batch_input)['heatmap'].numpy()
        time_end = time.time()
        
        delais = np.append(delais,time_end - time0)
        predictions.append(prediction)
        
        if i%10 == 0:
            print("Etape %d-%d moyenne : %4.1f ms" %(i,i+5,(delais[-10:].mean())*1000))

    print("Débit : %.0f images/s" %(nbr_run / delais.sum()))
    return predictions

In [ ]:
predictions = MesureDesPredictions(dataset_regression.unbatch(),infer,100)

### Evaluation de la précision

On évalue l'erreur du modèle sur la prédiction de l'ensemble du dataset utilisé pour entrainer le modèle :

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
erreurs = []

for images,heatmaps in dataset_regression.unbatch().take(10):
    images = tf.expand_dims(images,0)
    predict = infer(images)
    erreur = mse(predict['heatmap'],heatmaps)
    print(erreur.numpy())
    erreurs = np.append(erreurs,erreur.numpy())

print("Erreur moyenne : %f " %erreurs.mean())

### Redémarrage du kernel

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

# Modifications du réseau

TensorRT réalise plusieurs modifications et optimisations sur le réseau. Tout d'abord, les couches n'utilisant pas de sorties sont supprimées afin d'alléger les calculs. Ensuite, lorsque cela est possible, les couches de convolution, les offsets (bias) et les activations sont fusionnées pour ne former qu'une couche unique.  
La figure ci-dessous montre un exemple de **réseau de convolution non optimisé** :

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/Reseau_NonOptimise2.png?raw=true" width=600>

La figure ci-dessous montre le résultat obtenu après optimisation par **fusion des couches de manière verticale** (les couches fusionnées sont nommées **CBR**). Ceci permet d'améliorer les temps de calculs sur les GPUs car les opérations peuvent être faites sur des blocks parallélisés :

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/FusionVerticale_All.png?raw=true">

Un autre manière d'optimiser le réseau est de réaliser une **fusion horizontale des couches** qui prennent des entrées identiques :

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/FusionHorizontale_All.png?raw=true">

# Conversion du modèle vers un modèle TensorRT

Pour réaliser la converison du modèle, nous utilisons l'outil trtexec, en précisant le répertoire dans lequel le modèle est sauvegardé et les paramètres de conversion :  
- **precision_mode** : Format de codage des nombres : FP32, FP16 ou INT8. Les formats inférieurs au FP32 (FP16 et INT8) peuvent améliorer les performances des calculs. Le mode FP16 utilise des coeurs matériels avec des instructions sur des flottants 16bits lorsque cela est possible. Le mode INT8 utilise des coeurs matériels avec des instructions sur des entiers.     
- **max_batch_size** : Le batch-size maximum à utiliser pendant l'optimisation. Pendant l'excéution en temps réel, on peut choisir une valeur plus petite mais pas plus grande.  
- **minimum_segment_size** : Ce paramètre permet de préciser la valeur minimale de noeuds qu'il faut pour que la conversion du réseau soit exécutée. En conséquence, en général on choisit des valeurs inférieures à 5. Ce paramètre permet également de choisir le nombre minimum de noeuds pendant l'optimisation finale INT8 et donc d'optimiser la précision des résultats finaux.  
- **max_workspace_size_byte** : Les opérations d'optimisation de TensorRT ont besoin d'utiliser de l'espace de stockage temporaire. Ce paramètre permet de limiter l'espace maximal utilisé dans le GPU qu'une couche peut utiliser. Si une valeur insuffisante est donnée, TensorRT peut ne pas réussir à optimiser le modèle.

In [ ]:
!/usr/src/tensorrt/bin/trtexec

### Conversion au format FP32

In [ ]:
!/usr/src/tensorrt/bin/trtexec --workspace=256 --maxBatch=1 --onnx="/home/alexandre/Regression_Resnet18_saved_model/Regression_Resnet18_saved_model.onnx" --saveEngine="/home/alexandre/Regression_Resnet18_saved_model/Regression_Resnet18.engine"

# Utilisation du modèle TensorRT optimisé

### Chargement du moteur (engine)

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)

with open("Regression_Resnet18_saved_model/Regression_Resnet18.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

 ### Création du contexte

Création du contexte nécessaire pour lancer le modèle :

In [ ]:
context = engine.create_execution_context()

### Allocation de l'espace mémoire

Récupère la mémoire disponible du GPU :

In [ ]:
cuda.mem_get_info()

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/MemoireGPU.png?raw=true" width=600>

In [ ]:
for binding in engine:
    print(engine.get_binding_shape(binding))
    print(trt.nptype(engine.get_binding_dtype(binding)))

In [ ]:
# Allocation des buffers d'entrée / sortie dans la mémoire GPU
size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size

# Allocation de mémoire de type "page-locked" sur l'hôte
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))

# Allocation de mémoire dans la mémoire GPU
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)

In [ ]:
# Récupère les adresses en GPU des buffers entrées / sorties
bindings = [int(input_device_mem), int(output_device_mem)]

### Exécution d'une prédiction

Récupère l'image depuis le dataset :

In [ ]:
for image,heatmaps in dataset_regression.unbatch().take(1):
    image = np.asarray(image).astype(trt.nptype(PRECISION))       # (224,224,3)
    image = np.expand_dims(image,axis=0)                          # (1,224,224,3)
    np.copyto(input_host_mem,image.ravel())

Transfert les données de l'image vers la mémoire GPU (transfert Host => Device) :

In [ ]:
cuda.memcpy_htod(input_device_mem, input_host_mem)

Exécution du modèle :

In [ ]:
context.execute(batch_size=1,bindings=bindings)

In [ ]:
cuda.memcpy_dtoh(output_host_mem, output_device_mem)

In [ ]:
plt.imshow(np.reshape(input_host_mem,(224,224,3)).astype(np.uint8))

In [ ]:
plt.imshow(np.reshape(output_host_mem,(56,56)))

Libère la mémoire allouée :

In [ ]:
input_device_mem.free()
output_device_mem.free()
del input_host_mem
del output_host_mem
del context
del engine

# Analyse des performances du modèle optimisé

### Analyse de la vitesse de traitement

In [ ]:
import time

def MesureDesPredictions_TRT(dataset,nbr_run,PRECISION):
    delais = []
    predictions = []
    
    # Allocation des buffers d'entrée / sortie dans la mémoire GPU
    size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
    size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size

    # Allocation de mémoire de type "page-locked" sur l'hôte
    input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
    output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))

    # Allocation de mémoire dans la mémoire GPU
    input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
    output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)    

    # Récupère les adresses en GPU des buffers entrées / sorties
    bindings = [int(input_device_mem), int(output_device_mem)]
    
    # Transfert de l'mimage en mémoire sur le host
    for image,heatmaps in dataset.unbatch().take(1):
        image = np.asarray(image).astype(trt.nptype(PRECISION))
        image = np.expand_dims(image,axis=0)
        np.copyto(input_host_mem,image.ravel())
    
    # Initialisation des calculs
    print("Initialisation des calculs...")
    for i in range(5):
        cuda.memcpy_htod(input_device_mem, input_host_mem)
        context.execute(batch_size=1,bindings=bindings)        
        cuda.memcpy_dtoh(output_host_mem, output_device_mem)    
    
    # Lance les inférences
    for i in range(nbr_run):
        time0 = time.time()
        cuda.memcpy_htod(input_device_mem, input_host_mem)
        context.execute(batch_size=1,bindings=bindings)        
        cuda.memcpy_dtoh(output_host_mem, output_device_mem)    
        time_end = time.time()

        delais = np.append(delais,time_end - time0)

        predictions.append(output_host_mem)
        
        if i%10 == 0:
            print("Etape %d-%d moyenne : %4.1f ms" %(i,i+5,(delais[-10:].mean())*1000))
            
    # Libère la mémoire GPU
    input_device_mem.free()
    output_device_mem.free()
    
    # Libère la mémoire host
    del input_host_mem
    del output_host_mem

    print("Débit : %.0f images/s" %(nbr_run / delais.sum()))
    return predictions

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)

with open("Regression_Resnet18_saved_model/Regression_Resnet18.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
context = engine.create_execution_context()

In [ ]:
predictions = MesureDesPredictions_TRT(dataset_regression,100,PRECISION)

### Analyse de la précision du modèle optimisé

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)

with open("Regression_Resnet18_saved_model/Regression_Resnet18.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
context = engine.create_execution_context()

In [ ]:
erreurs = []

# Allocation des buffers d'entrée / sortie dans la mémoire GPU
size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size

# Allocation de mémoire de type "page-locked" sur l'hôte
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))

# Allocation de mémoire dans la mémoire GPU
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)    

# Récupère les adresses en GPU des buffers entrées / sorties
bindings = [int(input_device_mem), int(output_device_mem)]

for images,heatmaps in dataset_regression.unbatch().take(10):
    image = np.asarray(images).astype(trt.nptype(PRECISION))
    image = np.expand_dims(image,axis=0)
    np.copyto(input_host_mem,image.ravel())
    
    cuda.memcpy_htod(input_device_mem, input_host_mem)
    context.execute(batch_size=1,bindings=bindings)        
    cuda.memcpy_dtoh(output_host_mem, output_device_mem)    

    erreur = ((np.reshape(output_host_mem,(56,56,1)) - np.asarray(heatmaps))**2).mean(axis=None)
    print(erreur)
    erreurs = np.append(erreurs,erreur)

# Libère la mémoire GPU
input_device_mem.free()
output_device_mem.free()
    
# Libère la mémoire host
del input_host_mem
del output_host_mem
    
print("Erreur moyenne : %f " %erreurs.mean())

# Expérimentations avec le modèle optimisé

### Création de la classe Camera

In [ ]:
!ls -ltrh /dev/video*

In [ ]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        self.image = np.empty((self.display_height, self.display_width, 3), dtype=np.uint8)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
        return image_resized
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

In [ ]:
def InitCamera():
    camera = Camera(type_camera="USB",capture_device=0,
                capture_width=640,capture_height=480,
                display_width=224,display_height=224,
                fps=30,flip=0)
    return camera

In [ ]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

### Définition des catégories

In [ ]:
TACHE = "visage"
CATEGORIES = ['nez']

datasets = {}
for name in CATEGORIES:
    datasets[name] = []

### Fonction d'extraction des coordonnées

In [ ]:
import tensorflow as tf
from tensorflow import keras

def GetCoordFromHeatmap(heatmap,sampling=4):
    hmax = tf.keras.layers.MaxPooling2D(3, 1, padding="same")(heatmap)
    keep = tf.cast(tf.equal(heatmap, hmax), tf.float32)
    prod = hmax*keep

    # Applatissement
    prod_applati = tf.reshape(prod, (1, -1))
    
    # Récupère les index du maximum
    scores, index = tf.nn.top_k(prod_applati, k=1)
    
    # Calcul des coordonnées
    xs = tf.cast(index % heatmap[0,:,:,0].shape[1], tf.float32)
    ys = tf.cast(index // heatmap[0,:,:,0].shape[0], tf.float32)
    return xs*sampling,ys*sampling

### Interface d'acquisition en temps réel

In [ ]:
%config IPCompleter.greedy=True

import numpy as np
import cv2
import os
import shutil
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda

class ThreadLive(threading.Thread):
    def __init__(self, state_widget, category_widget, preview_widget, camera, fichier_engine, PRECISION):
        threading.Thread.__init__(self)

        self.cfx = cuda.Device(0).make_context()
        
        TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
        runtime = trt.Runtime(TRT_LOGGER)

        # Chargement du moteur
        print("Chargement du moteur...")
        with open(fichier_engine, 'rb') as f:
            buf = f.read()
            engine = runtime.deserialize_cuda_engine(buf)

        # Création du context
        print("Création du context...")
        context = engine.create_execution_context()

        # Allocation des buffers d'entrée / sortie dans la mémoire GPU
        print("Allocation de la mémoire ...")
        size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
        size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size
        input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
        output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))
        input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
        output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)    
        bindings = [int(input_device_mem), int(output_device_mem)]        
        print("Modèle initialisé ...")
        
        # Sauvegarde dand les variables internes de la classe
        self.context = context
        self.engine  = engine
        self.input_host_mem = input_host_mem
        self.output_host_mem = output_host_mem
        self.input_device_mem = input_device_mem
        self.output_device_mem = output_device_mem
        self.bindings = bindings
        self.PRECISION = PRECISION
        self.state_widget = state_widget
        self.category_widget = category_widget
        self.camera = camera
        self.preview_widget = preview_widget
        
    def run(self):
        context = self.context
        input_host_mem = self.input_host_mem
        output_host_mem = self.output_host_mem
        input_device_mem = self.input_device_mem
        output_device_mem = self.output_device_mem
        bindings = self.bindings
        PRECISION = self.PRECISION
        state_widget = self.state_widget
        camera = self.camera
        category_widget = self.category_widget
        preview_widget = self.preview_widget
        PRECISION = self.PRECISION
        
        while state_widget.value == 'live':
            # Capture de l'image
            image_camera = camera.image

            # Récupération de la catégorie
            categorie = CATEGORIES.index(category_widget.value)

            # Prédiciton de la heatmap
            image = np.asarray(image_camera).astype(trt.nptype(PRECISION))
            image = np.expand_dims(image,axis=0)
            np.copyto(input_host_mem,image.ravel())

            self.cfx.push()
            cuda.memcpy_htod(input_device_mem, input_host_mem)
            context.execute(batch_size=1,bindings=bindings) 
            cuda.memcpy_dtoh(output_host_mem, output_device_mem)    
            self.cfx.pop()
            
            # Extraction des coordonnées
            heatmap = np.reshape(output_host_mem,(1,56,56,1))
            x,y = GetCoordFromHeatmap(heatmap)

            # Affichage de l'image avec les coordonnées
            prediction = image_camera.copy()
            prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
            preview_widget.value = bgr8_to_jpeg(prediction)

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display

fichier_engine = "Regression_Resnet18_saved_model/Regression_Resnet18.engine"
PRECISION = trt.float32

# Initialise la caméra
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass

camera = InitCamera()

# Création du widget de la vidéo
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Lancement de la vidéo
camera.capture_video(run=True)
camera_link = traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Création des widgets
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='Catégorie')
preview_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))

def start_live(change):
    if change['new'] == 'live':
        global execute_thread
        execute_thread = ThreadLive(state_widget, category_widget, preview_widget, camera, fichier_engine, PRECISION)
        execute_thread.start()
    else:
        execute_thread.join()
       
state_widget.observe(start_live, names='value')

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget,preview_widget]),category_widget,state_widget])


display(data_collection_widget)